In [1]:
import keras
# The line below imports backend module from keras configuration. This makes this code compatible with theano and tensorflow.
from keras import backend as K
# The imports below are the keras API's which will help us to build, preprocess and infer
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
# Numpy is required for feeding data to model
import numpy as np
# The import below is used to visualize our image
from IPython.display import Image

Using TensorFlow backend.


In [2]:
from keras.callbacks import ModelCheckpoint

In [3]:
HEIGHT = 64
WIDTH = 64

In [4]:
base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Instructions for updating:
Colocations handled automatically by placer.


c:\users\inpatian\appdata\local\programs\python\python36\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [5]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
preds=Dense(9,activation='softmax')(x)

In [6]:
model=Model(inputs=base_model.input,outputs=preds)

In [7]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [8]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 pool1_pad
6 max_pooling2d_1
7 res2a_branch2a
8 bn2a_branch2a
9 activation_2
10 res2a_branch2b
11 bn2a_branch2b
12 activation_3
13 res2a_branch2c
14 res2a_branch1
15 bn2a_branch2c
16 bn2a_branch1
17 add_1
18 activation_4
19 res2b_branch2a
20 bn2b_branch2a
21 activation_5
22 res2b_branch2b
23 bn2b_branch2b
24 activation_6
25 res2b_branch2c
26 bn2b_branch2c
27 add_2
28 activation_7
29 res2c_branch2a
30 bn2c_branch2a
31 activation_8
32 res2c_branch2b
33 bn2c_branch2b
34 activation_9
35 res2c_branch2c
36 bn2c_branch2c
37 add_3
38 activation_10
39 res3a_branch2a
40 bn3a_branch2a
41 activation_11
42 res3a_branch2b
43 bn3a_branch2b
44 activation_12
45 res3a_branch2c
46 res3a_branch1
47 bn3a_branch2c
48 bn3a_branch1
49 add_4
50 activation_13
51 res3b_branch2a
52 bn3b_branch2a
53 activation_14
54 res3b_branch2b
55 bn3b_branch2b
56 activation_15
57 res3b_branch2c
58 bn3b_branch2c
59 add_5
60 activation_16
61 res3c_branch2a
62 bn3c_branch2a

In [9]:
for layer in model.layers[:171]:
    layer.trainable=False

for layer in model.layers[171:]:
    layer.trainable=True

In [10]:
for i,layer in enumerate(model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 conv1_pad False
2 conv1 False
3 bn_conv1 False
4 activation_1 False
5 pool1_pad False
6 max_pooling2d_1 False
7 res2a_branch2a False
8 bn2a_branch2a False
9 activation_2 False
10 res2a_branch2b False
11 bn2a_branch2b False
12 activation_3 False
13 res2a_branch2c False
14 res2a_branch1 False
15 bn2a_branch2c False
16 bn2a_branch1 False
17 add_1 False
18 activation_4 False
19 res2b_branch2a False
20 bn2b_branch2a False
21 activation_5 False
22 res2b_branch2b False
23 bn2b_branch2b False
24 activation_6 False
25 res2b_branch2c False
26 bn2b_branch2c False
27 add_2 False
28 activation_7 False
29 res2c_branch2a False
30 bn2c_branch2a False
31 activation_8 False
32 res2c_branch2b False
33 bn2c_branch2b False
34 activation_9 False
35 res2c_branch2c False
36 bn2c_branch2c False
37 add_3 False
38 activation_10 False
39 res3a_branch2a False
40 bn3a_branch2a False
41 activation_11 False
42 res3a_branch2b False
43 bn3a_branch2b False
44 activation_12 False
45 res3a_branch2c False

In [11]:
TRAIN_DIR = "F:\\AI\\to_explore\\keras_open_source_projects\\NEU-CLS-64"
BATCH_SIZE = 16

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

In [12]:
train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   class_mode='categorical',
                                                   color_mode='rgb')

Found 7226 images belonging to 9 classes.


In [13]:
model_checkpoint = ModelCheckpoint('neu_cls_two_layers.h5', monitor='loss',verbose=1, save_best_only=True)

In [14]:
adam = Adam(lr=0.001)

In [15]:
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
step_size_train=train_generator.n//train_generator.batch_size

In [17]:
model.fit_generator(train_generator,steps_per_epoch=step_size_train,epochs=100,callbacks=[model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
451/451 [==============================] - 141s 313ms/step - loss: 0.6952 - acc: 0.7946

Epoch 00001: loss improved from inf to 0.69518, saving model to neu_cls_two_layers.h5
Epoch 2/100
451/451 [==============================] - 149s 330ms/step - loss: 0.4389 - acc: 0.8595

Epoch 00002: loss improved from 0.69518 to 0.43886, saving model to neu_cls_two_layers.h5
Epoch 3/100
451/451 [==============================] - 152s 337ms/step - loss: 0.3701 - acc: 0.8813

Epoch 00003: loss improved from 0.43886 to 0.37025, saving model to neu_cls_two_layers.h5
Epoch 4/100
451/451 [==============================] - 153s 339ms/step - loss: 0.3437 - acc: 0.8855

Epoch 00004: loss improved from 0.37025 to 0.34335, saving model to neu_cls_two_layers.h5
Epoch 5/100
451/451 [==============================] - 154s 341ms/step - loss: 0.3371 - acc: 0.8883

Epoch 00005: loss improved from 0.34335 to 0.33692, saving model to neu_cls_two_layers.h5
E

Epoch 49/100
451/451 [==============================] - 157s 348ms/step - loss: 0.2063 - acc: 0.9309

Epoch 00049: loss did not improve from 0.19145
Epoch 50/100
451/451 [==============================] - 160s 355ms/step - loss: 0.2143 - acc: 0.9310

Epoch 00050: loss did not improve from 0.19145
Epoch 51/100
451/451 [==============================] - 159s 353ms/step - loss: 0.2036 - acc: 0.9338

Epoch 00051: loss did not improve from 0.19145
Epoch 52/100
451/451 [==============================] - 157s 347ms/step - loss: 0.1952 - acc: 0.9336

Epoch 00052: loss did not improve from 0.19145
Epoch 53/100
451/451 [==============================] - 157s 349ms/step - loss: 0.2054 - acc: 0.9315

Epoch 00053: loss did not improve from 0.19145
Epoch 54/100
451/451 [==============================] - 158s 350ms/step - loss: 0.2184 - acc: 0.9319

Epoch 00054: loss did not improve from 0.19145
Epoch 55/100
451/451 [==============================] - 157s 349ms/step - loss: 0.1926 - acc: 0.9356

Epoc